In [141]:
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
from torchvision import transforms
import torch
import numpy as np
from collections import Counter

In [142]:
class ImageDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.folder = folder
        self.transform = transform
        self.labels = []
        self.image_paths = []
        self.label_map = {'Cat':0, 'Dog':1}

        for label in ['Cat', 'Dog']:
            label_path = os.path.join(folder, label)
            for img in os.listdir(label_path):
                if img is not None:
                    self.image_paths.append(os.path.join(label_path, img))
                    self.labels.append(self.label_map[label])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image = self.image_paths[index]
        image = Image.open(image)
        label = self.labels[index]

        if self.transform:
            image = self.transform(image)

        return image, label

In [143]:
train = "C://large_files//DogCat//train"
test = "C://large_files//DogCat//test"

img_size = (64, 64)

transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


train_dataset = ImageDataset(train, transform)
test_dataset = ImageDataset(test, transform)

In [144]:
image, label = train_dataset.__getitem__(600)

image.shape, label

(torch.Size([3, 64, 64]), 1)

In [145]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [146]:
for image, label in train_dataloader:
    print(image.shape)
    print(label.shape)
    break

torch.Size([32, 3, 64, 64])
torch.Size([32])


In [147]:
for data, _ in test_dataloader:
    print(data.shape)
    for x in data:
        print(x.shape)
        break
    break

torch.Size([32, 3, 64, 64])
torch.Size([3, 64, 64])


In [148]:
class KNN:
    def __init__(self, k=3):
        self.train_labels = None
        self.train_imgs = None
        self.k = k
        
    def train(self, train_dataloader):
        self.train_imgs = []
        self.train_labels = []

        for imgs, labels in train_dataloader:
            self.train_imgs.append(imgs)
            self.train_labels.append(labels)

        self.train_imgs = torch.cat(self.train_imgs)
        self.train_labels = torch.cat(self.train_labels)

        self.train_imgs = self.train_imgs.view(self.train_imgs.size(0), -1)
        print("Done with train.")

    def predict(self, test_dataloader):
        predictions = []
        actual_labels = []
        for imgs, labels in test_dataloader:
            imgs = imgs.view(imgs.size(0), -1)
            for img in imgs:
                predictions.append(self._predict(img))
            actual_labels.extend(labels)

        return np.array(predictions), np.array(actual_labels)

    def _predict(self, img):
        distances = torch.sqrt(torch.sum((self.train_imgs - img) ** 2, dim=1))

        k_indices = distances.argsort()[:self.k]

        k_nearest_labels = [self.train_labels[i].item() for i in k_indices]

        most_common = Counter(k_nearest_labels).most_common(1)

        return most_common[0][0]



In [149]:
# from sklearn.model_selection import cross_val_score, KFold
# from sklearn.neighbors import KNeighborsClassifier
# from torchvision import datasets
# import numpy as np
#
# data_dir = "C://large_files//DogCat"
# dataset = datasets.ImageFolder(data_dir, transform=transform)
#
# features, labels = list(), list()
# for img, label in dataset:
#     flattened = img.view(-1)
#     features.append(flattened)
#     labels.append(label)
#
# features, labels = np.array(features), np.array(labels)
#
# features = features.astype(np.float32)
# labels = labels.astype(np.int64)
#
#
# knn = KNeighborsClassifier(n_neighbors=3)
#
# kfold = KFold(n_splits=5, shuffle=True, random_state=42)
#
# cv_scores = cross_val_score(knn, features, labels, cv=kfold)
#
#
# print(f"Cross-Validation Scores: {cv_scores}")
# print(f"Mean Accuracy: {np.mean(cv_scores)}")

In [150]:
def calculate_accuracy(predictions, labels):
    correct = np.sum(predictions == labels)
    total = len(labels)
    accuracy = correct / total
    return accuracy

In [151]:
knn = KNN(k=5)
knn.train(train_dataloader)
predictions, actual_labels = knn.predict(test_dataloader)

print(f"Accuracy is {calculate_accuracy(predictions, actual_labels)}")

Done with train.
Accuracy is 0.59
